# Carga de librerías necesarias

In [ ]:
import pandas as pd, numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random
import time
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# Carga de dataset

In [ ]:
data = pd.read_csv('mushrooms.csv')
data.head(15)

In [ ]:
att = data.columns
print(att)

# Cambio de nombres de atributos

In [ ]:
att = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
       'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color',
       'stalk-shape', 'stalk-root', 'stalk-surface-above-ring',
       'stalk-surface-below-ring', 'stalk-color-above-ring',
       'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number',
       'ring-type', 'spore-print-color', 'population', 'habitat', 'class']

data = data[att]
data_new = data.copy()
data_mode = data.copy()
data_HD = data.copy()
data

In [ ]:
data.info()

# Datos por atributo

In [ ]:
dat_miss=[]
for n,col in enumerate(att):
    dat_miss+=[[n,col,data[col].unique().tolist()]]
#     print(data[col].unique(),col)
    
print(tabulate(dat_miss,headers=['#','Atributo','Datos']))

# Cuantificación de valores por atributo

In [ ]:
for col in data_new.columns:
    plt.figure()
    plt.title('Distribución de datos '+col)
    plt.grid('on')
    sns.countplot(data[col])
    plt.show()

# Codificación para obtención de mapa de correlación

In [ ]:
for d in dat_miss:
#     print(d[1])
    for code,datum in enumerate(d[2]):
#         print(datum)
        data_new.loc[data_new[d[1]]==datum,d[1]]=code
    data_new[d[1]] = data_new[d[1]].astype('float')

# Dataset codificado

In [ ]:
data_new.head(15)

# Mapa de correlación

In [ ]:
corr_data = data_new.corr(method = 'pearson')

plt.figure(figsize= (30, 30),dpi=150)
plt.title('Correlación de atributos de Mushroom', fontsize = 30)
sns.heatmap(corr_data, annot = True, cmap = 'Blues', fmt=".2f")

plt.show()

# Datos por atributo despues de correlación

In [ ]:
dat_miss_cods=[]
for n,col in enumerate(att):
    dat_miss_cods += [[n,col,data_new[col].unique().tolist()]]
#     print(data[col].unique(),col)
    
print(tabulate(dat_miss_cods,headers=['#','Atributo','Datos']))

# Imputacion de datos por media (moda) por clase

In [ ]:
# media por clase 0
indices=[]
for ins in tqdm(data.index):
#     print(ins,n,data.loc[ins,data.columns[11]])
    if 'p' == data.loc[ins,data.columns[-1]]:
        indices += [ins]
        
data_class0 = data.loc[indices,:]
# data_missings

# media por clase 1
indices=[]
for ins in tqdm(data.index):
#     print(ins,n,data.loc[ins,data.columns[11]])
    if 'e' == data.loc[ins,data.columns[-1]]:
        indices += [ins]
        
data_class1 = data.loc[indices,:]

# deteccion de datos faltantes para imputación
indices=[]
for ins in tqdm(data.index):
#     print(ins,n,data.loc[ins,data.columns[11]])
    if '?' == data.loc[ins,data.columns[10]]:
        indices += [ins]
        
data_missings = data.loc[indices,:]
data_missings

# Conteo de atributo con datos perdidos antes de imputación

In [ ]:
print(data_class0['stalk-root'].value_counts())
print(data_class1['stalk-root'].value_counts())

# Imputación de datos

In [ ]:
dat0 = data_class0['stalk-root'].mode().tolist()
print('Moda de atributo stalk-root para clase 0: ',dat0)
dat1 = data_class1['stalk-root'].mode().tolist()
print('Moda de atributo stalk-root para clase 1: ',dat1)

data_class0.loc[data_class0['stalk-root']=='?','stalk-root']=dat0[0]
data_class1.loc[data_class1['stalk-root']=='?','stalk-root']=dat0[0]

# Comprobación de datos perdidos despues de imputación de datos

In [ ]:
print(data_class0['stalk-root'].value_counts())
print(data_class1['stalk-root'].value_counts())

# Concatenación de datos completos e imputados
data_mode = pd.concat([data_class0, data_class1])

# Comprobación de datos por atributo y no nulos

In [ ]:
dat_miss_cods=[]
for n,col in enumerate(att):
    dat_miss_cods += [[n,col,data_mode[col].unique().tolist()]]
#     print(data[col].unique(),col)
    
print(tabulate(dat_miss_cods,headers=['#','Atributo','Datos']))

# Conteo de datos despues de imputación de datos faltantes

In [ ]:
plt.figure()
plt.title('Distribución de datos '+col)
plt.grid('on')
sns.countplot(data_mode['stalk-root'])
plt.show()

# Entrenamiento con datos imputados por moda de clases
## Muestreo de datos de ejemplo para oob (validación cruzada)

In [ ]:
data_mode.sample(frac=2/3,replace=True)

# Codificación de datos para train a partir de imputación por moda

In [ ]:
data_mode_cod= data_mode.copy()
for d in dat_miss_cods:
    for code,datum in enumerate(d[2]):
        data_mode_cod.loc[data_mode_cod[d[1]]==datum,d[1]]=code
    data_mode_cod[d[1]] = data_mode_cod[d[1]].astype('float')
data_mode_cod.sort_index().head(15)

# División de datos en train y test 80/20

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, KFold

# Aleatorización de datos
x = data_mode_cod[data_mode_cod.columns[:-1]]
y = data_mode_cod[data_mode_cod.columns[-1]]
print(x.shape)
print(y.shape)
train, labels = shuffle(x, y)

# Division de datos en train y test
x_train, x_test, y_train, y_test = train_test_split(train, labels, train_size = 0.8)
print(x_train.shape)
print(y_test.shape)
print(data_mode_cod.columns[:-1])
print(data_mode[data_mode.columns[-1]].unique().tolist())

# Train de Modelo RandomForestClassifier con datos imputados por moda por clase

In [ ]:
from sklearn.ensemble import RandomForestClassifier

bosque = RandomForestClassifier(n_estimators=100,
                                   criterion='entropy',
                                   max_features='sqrt',
                                   bootstrap=True,
                                   max_samples=2/3,
                                   oob_score=True)

bosque.fit(x_train, y_train)

# Predicción y score con datos train y test

In [ ]:
test0 = x_test.iloc[0].tolist()
test1 = x_test.iloc[1].tolist()
test2 = x_test.iloc[2].tolist()
test3 = x_test.iloc[3].tolist()
test4 = x_test.iloc[4].tolist()

print('Predicción: ', bosque.predict([test0]))
print('Verdadero: ', y_test.iloc[0].tolist())
print('Predicción: ', bosque.predict([test1]))
print('Verdadero: ', y_test.iloc[1].tolist())
print('Predicción: ', bosque.predict([test2]))
print('Verdadero: ', y_test.iloc[2].tolist())
print('Predicción: ', bosque.predict([test3]))
print('Verdadero: ', y_test.iloc[3].tolist())
print('Predicción: ', bosque.predict([test4]))
print('Verdadero: ', y_test.iloc[4].tolist())

print('\nscore con datos de train',bosque.score(x_train,y_train))
print('score con datos de test',bosque.score(x_test,y_test))
print('score',bosque.oob_score_)

# Reporte y matriz de confusión¶

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred_test = bosque.predict(x_test)
print('Reporte y matriz de confusión')
print(accuracy_score(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

In [ ]:
matrix = confusion_matrix(y_test, y_pred_test)

# Build the plot
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Greens, linewidths=0.2,fmt='d')

# Add labels to the plot
class_names = ['poisonous', 'edible']
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=25,fontsize=20)
plt.yticks(tick_marks2, class_names, rotation=0,fontsize=20)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix for Random Forest Model')
plt.show()

# Visualización de Bosque Aleatorio

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree

for e,arbol in zip(range(10),bosque.estimators_):
    fig = plt.figure(figsize=(40,35),dpi=300)
    plt.title('Arbol #: ' + str(e))
    tree.plot_tree(arbol, 
                   feature_names=data_mode_cod.columns[:-1],
                   class_names=class_names,
                  filled=True)
#     plt.savefig('Arbol_' + str(e)+'.png', dpi=300)
    plt.show()

# Evolución del out-of-bag-error vs número árboles

In [ ]:
# Validación empleando el Out-of-Bag error
# ==============================================================================
train_scores = []
oob_scores   = []

# Valores evaluados
estimator_range = range(1, 50, 5)

# Bucle para entrenar un modelo con cada valor de n_estimators y extraer su error
# de entrenamiento y de Out-of-Bag.
for n_estimators in estimator_range:
    bosque = RandomForestClassifier(
                n_estimators = n_estimators,
                criterion    = 'entropy',
                max_depth    = None,
                max_features = 'sqrt',
                oob_score    = True,
                n_jobs       = -1,
                random_state = 123
             )
    bosque.fit(x_train, y_train)
    train_scores.append(bosque.score(x_train, y_train))
    oob_scores.append(bosque.oob_score_)
    
# Gráfico con la evolución de los errores
fig, ax = plt.subplots(figsize=(6, 3.84))
ax.plot(estimator_range, train_scores, label="train scores")
ax.plot(estimator_range, oob_scores, label="out-of-bag scores")
ax.plot(estimator_range[np.argmax(oob_scores)], max(oob_scores),
        marker='o', color = "red", label="max score")
ax.set_ylabel("R^2")
ax.set_xlabel("n_estimators")
ax.set_title("Evolución del out-of-bag-error vs número árboles")
plt.legend();
print(f"Valor óptimo de n_estimators: {estimator_range[np.argmax(oob_scores)]}")

# Evaluacion del cv-error vs número de arboles

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# Validación empleando k-cross-validation y neg_root_mean_squared_error
# ==============================================================================
train_scores = []
cv_scores    = []

# Valores evaluados
estimator_range = range(1, 50, 5)

# Bucle para entrenar un modelo con cada valor de n_estimators y extraer su error
# de entrenamiento y de k-cross-validation.
for n_estimators in estimator_range:
    
    bosque = RandomForestClassifier(
                n_estimators = n_estimators,
                criterion    = 'entropy',
                max_depth    = None,
                max_features = 'sqrt',
                oob_score    = False,
                n_jobs       = -1,
                random_state = 123
             )
    
    # Error de train
    bosque.fit(x_train, y_train)
    predicciones = bosque.predict(x_train)
    rmse = mean_squared_error(
            y_true  = y_train,
            y_pred  = predicciones,
            squared = False
           )
    train_scores.append(rmse)
    
    # Error de validación cruzada
    scores = cross_val_score(
                estimator = bosque,
                X         = x_train,
                y         = y_train,
                scoring   = 'neg_root_mean_squared_error',
                cv        = 5
             )
    # Se agregan los scores de cross_val_score() y se pasa a positivo
    cv_scores.append(-1*scores.mean())
    
# Gráfico con la evolución de los errores
fig, ax = plt.subplots(figsize=(6, 3.84))
ax.plot(estimator_range, train_scores, label="train scores")
ax.plot(estimator_range, cv_scores, label="cv scores")
ax.plot(estimator_range[np.argmin(cv_scores)], min(cv_scores),
        marker='o', color = "red", label="min score")
ax.set_ylabel("root_mean_squared_error")
ax.set_xlabel("n_estimators")
ax.set_title("Evolución del cv-error vs número árboles")
plt.legend();
print(f"Valor óptimo de n_estimators: {estimator_range[np.argmin(cv_scores)]}")

# 2.- Imputación Hot-Deck
## Detección de datos perdidos

In [ ]:
indices=[]
for ins in tqdm(data.index):
#     print(ins,n,data.loc[ins,data.columns[11]])
    if '?' == data.loc[ins,data.columns[10]]:
        indices += [ins]
        
data_missings = data.loc[indices,:]
data_missings

# Detección de datos validos

In [ ]:
# detección de datos no perdidos (missings) por indice en database
indices=[]
for ins in tqdm(data.index):
#     print(ins,n,data.loc[ins,data.columns[11]])
    if '?' != data.loc[ins,data.columns[10]]:
        indices += [ins]
        
data_fill = data.loc[indices,:]

# Toma de datos validos del atributo con datos perdidos para su uso en completar los valores faltantes

In [ ]:
datos_vals=[]
for i in data_fill.index:
    datum = data_fill.loc[i,data_fill.columns[10]]
    datos_vals += [datum]

# Selección de dato aleatorio para comenzar a rellenar datos faltantes

In [ ]:
item_ini = random.choice(datos_vals)

part1= datos_vals[:datos_vals.index(item_ini)]
part2= datos_vals[datos_vals.index(item_ini):]

ht_deck= part1+part2

# Comprobación de datos antes de imputación

In [ ]:
print(data_missings['stalk-root'].value_counts())
print(data_fill['stalk-root'].value_counts())

# Imputación de datos faltantes mediante Hot-Deck

In [ ]:
for i,n in zip(data_missings.index,ht_deck):
    data_missings.loc[i,data_missings.columns[10]] = n

# Comprobación de datos después de imputación

In [ ]:
print(data_missings['stalk-root'].value_counts())
print(data_fill['stalk-root'].value_counts())

# Creación de dataframe con datos imputados

In [ ]:
data_Hot_Deck = pd.concat([data_fill,data_missings])
data_Hot_Deck

# Comprobación de datos por atributo y no nulos

In [ ]:
dat_miss_cods=[]
for n,col in enumerate(att):
    dat_miss_cods += [[n,col,data_Hot_Deck[col].unique().tolist()]]
#     print(data[col].unique(),col)
    
print(tabulate(dat_miss_cods,headers=['#','Atributo','Datos']))

# Conteo de datos en atributo con datos imputados

In [ ]:
plt.figure()
plt.title('Distribución de datos '+col)
plt.grid('on')
sns.countplot(data_Hot_Deck['stalk-root'])
plt.show()

# Codificación de datos para train a partir de imputación Hot-Deck

In [ ]:
data_Hot_Deck_cod= data_Hot_Deck.copy()
for d in dat_miss_cods:
    for code,datum in enumerate(d[2]):
        data_Hot_Deck_cod.loc[data_Hot_Deck_cod[d[1]]==datum,d[1]]=code
    data_Hot_Deck_cod[d[1]] = data_Hot_Deck_cod[d[1]].astype('float')
data_Hot_Deck_cod

# División de datos en train y test 80/20

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, KFold

# Aleatorización de datos
x = data_Hot_Deck_cod[data_Hot_Deck_cod.columns[:-1]]
y = data_Hot_Deck_cod[data_Hot_Deck_cod.columns[-1]]
print(x.shape)
print(y.shape)
train, labels = shuffle(x, y)

# Division de datos en train y test
x_train, x_test, y_train, y_test = train_test_split(train, labels, train_size = 0.8)
x_train.shape
print(data_Hot_Deck_cod.columns[:-1])
print(data_Hot_Deck[data_Hot_Deck.columns[-1]].unique().tolist())

# Train de Modelo RandomForestClassifier con datos imputados por moda por clase

In [ ]:
from sklearn.ensemble import RandomForestClassifier

bosque = RandomForestClassifier(n_estimators=100,
                                   criterion='entropy',
                                   max_features='sqrt',
                                   bootstrap=True,
                                   max_samples=2/3,
                                   oob_score=True)

bosque.fit(x_train, y_train)

# Predicción y score con datos train y test

In [ ]:
test0 = x_test.iloc[0].tolist()
test1 = x_test.iloc[1].tolist()
test2 = x_test.iloc[2].tolist()
test3 = x_test.iloc[3].tolist()
test4 = x_test.iloc[4].tolist()

print('Prediccion: ', bosque.predict([test0]))
print('Verdadero: ', y_test.iloc[0].tolist())
print('Prediccion: ', bosque.predict([test1]))
print('Verdadero: ', y_test.iloc[1].tolist())
print('Prediccion: ', bosque.predict([test2]))
print('Verdadero: ', y_test.iloc[2].tolist())
print('Prediccion: ', bosque.predict([test3]))
print('Verdadero: ', y_test.iloc[3].tolist())
print('Prediccion: ', bosque.predict([test4]))
print('Verdadero: ', y_test.iloc[4].tolist())

print('\nscore con datos de train',bosque.score(x_train,y_train))
print('score con datos de test',bosque.score(x_test,y_test))
print('score',bosque.oob_score_)

# Reporte y matriz de confusión

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_pred_test = bosque.predict(x_test)
print('Matriz de confusión y reporte')
print(accuracy_score(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

In [ ]:
matrix = confusion_matrix(y_test, y_pred_test)

# Build the plot
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Greens, linewidths=0.2,fmt='d')

# Add labels to the plot
class_names = ['poisonous', 'edible']
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=25,fontsize=20)
plt.yticks(tick_marks2, class_names, rotation=0,fontsize=20)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix for Random Forest Model')
plt.show()

# Visualización de Bosque Aleatorio

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree

for e,arbol in zip(range(10),bosque.estimators_):
    fig = plt.figure(figsize=(40,35),dpi=300)
    plt.title('Arbol #: ' + str(e))
    tree.plot_tree(arbol, 
                   feature_names=data_Hot_Deck_cod.columns[:-1],
                   class_names=class_names,
                  filled=True)
#     plt.savefig('Arbol_HD_' + str(e)+'.png', dpi=300)
    plt.show()

# Evolución del OOB Error vs Número árboles

In [ ]:
# Validación empleando el Out-of-Bag error
# ==============================================================================
train_scores = []
oob_scores   = []

# Valores evaluados
estimator_range = range(1, 50, 5)

# Bucle para entrenar un modelo con cada valor de n_estimators y extraer su error
# de entrenamiento y de Out-of-Bag.
for n_estimators in estimator_range:
    bosque = RandomForestClassifier(
                n_estimators = n_estimators,
                criterion    = 'entropy',
                max_depth    = None,
                max_features = 'sqrt',
                oob_score    = True,
                n_jobs       = -1,
                random_state = 123
             )
    bosque.fit(x_train, y_train)
    train_scores.append(bosque.score(x_train, y_train))
    oob_scores.append(bosque.oob_score_)
    
# Gráfico con la evolución de los errores
fig, ax = plt.subplots(figsize=(6, 3.84))
ax.plot(estimator_range, train_scores, label="train scores")
ax.plot(estimator_range, oob_scores, label="out-of-bag scores")
ax.plot(estimator_range[np.argmax(oob_scores)], max(oob_scores),
        marker='o', color = "red", label="max score")
ax.set_ylabel("R^2")
ax.set_xlabel("n_estimators")
ax.set_title("Evolución del out-of-bag-error vs número árboles")
plt.legend();
print(f"Valor óptimo de n_estimators: {estimator_range[np.argmax(oob_scores)]}")

# Evolución del cv-error vs número árboles

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# Validación empleando k-cross-validation y neg_root_mean_squared_error
# ==============================================================================
train_scores = []
cv_scores    = []

# Valores evaluados
estimator_range = range(1, 50, 5)

# Bucle para entrenar un modelo con cada valor de n_estimators y extraer su error
# de entrenamiento y de k-cross-validation.
for n_estimators in estimator_range:
    
    bosque = RandomForestClassifier(
                n_estimators = n_estimators,
                criterion    = 'entropy',
                max_depth    = None,
                max_features = 'sqrt',
                oob_score    = False,
                n_jobs       = -1,
                random_state = 123
             )
    
    # Error de train
    bosque.fit(x_train, y_train)
    predicciones = bosque.predict(x_train)
    rmse = mean_squared_error(
            y_true  = y_train,
            y_pred  = predicciones,
            squared = False
           )
    train_scores.append(rmse)
    
    # Error de validación cruzada
    scores = cross_val_score(
                estimator = bosque,
                X         = x_train,
                y         = y_train,
                scoring   = 'neg_root_mean_squared_error',
                cv        = 5
             )
    # Se agregan los scores de cross_val_score() y se pasa a positivo
    cv_scores.append(-1*scores.mean())
    
# Gráfico con la evolución de los errores
fig, ax = plt.subplots(figsize=(6, 3.84))
ax.plot(estimator_range, train_scores, label="train scores")
ax.plot(estimator_range, cv_scores, label="cv scores")
ax.plot(estimator_range[np.argmin(cv_scores)], min(cv_scores),
        marker='o', color = "red", label="min score")
ax.set_ylabel("root_mean_squared_error")
ax.set_xlabel("n_estimators")
ax.set_title("Evolución del cv-error vs número árboles")
plt.legend();
print(f"Valor óptimo de n_estimators: {estimator_range[np.argmin(cv_scores)]}")